## [미니프로젝트] 악성사이트 탐지 머신러닝 모델 개발

### 여러분은 기업 보안팀에서 근무중인 엔지니어로써, 웹페이지에서 추출한 Feature(특징) 기반으로 악성사이트를 탐지하는 머신러닝 모델 개발 미션을 부여받았습니다.

### ▣ 우리가 풀어야 하는 문제는 무엇인가요?
 - 웹 페이지에서 Feature를 추출하세요.
 - 악성사이트 여부를 판별하는 성능 좋은 AI모델을 생성하세요.

<br>

---

## ▣ 데이터 소개
* 웹 크롤링 데이터셋 : Feature_Website.xlsx

## ▣ 웹 크롤링 데이터셋의 변수 소개
* html_code : 크롤링을 활용해 수집한 HTML Code 원본
* repu : 악성사이트 여부 (malicious : 악성사이트, benign : 정상사이트)
<br>

---

## <b>[1단계] 데이터 수집</b>

* 1단계에서는 크롤링으로 수집한 HTML Code를 활용해 Feature를 만드는 과정을 체험합니다.

# <b>Step 0. 본격적인 실습 전 packages 설치
* Beautifulsoup 라이브러리 설치
* openpyxl 라이브러리 설치

In [148]:
!pip install beautifulsoup4
!pip install openpyxl

* 데이터 프레임 관련 라이브러리 Import

In [149]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup

---
## <b>Q1. 데이터 불러오기
### 정상/악성 HTML Code가 저장된 엑셀파일 불러오기
- 파일명 : Feature Website.xlsx


### <span style="color:darkred">[문제1] Pandas 라이브러리를 활용해서 'Feature Website.xlsx'파일을 'df' 변수에 저장하고 그 info()및 head()를 통해 데이터를 확인하세요.<span>

In [150]:
# 아래에 실습코드를 작성하고 결과를 확인합니다.
df = pd.read_excel('Feature_website.xlsx')

In [151]:
# 데이터 프레임의 info를 확인합니다.
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   html_code  40 non-null     object
 1   repu       40 non-null     object
dtypes: object(2)
memory usage: 768.0+ bytes


In [152]:
# 불러온 데이터를 확인합니다.
df.head()

,html_code,repu
0,<!DOCTYPE html>\n<!--[if lt IE 7]> <html lang=...,malicious
1,\n\t\n\n\n\t\n\n\t\n\n\n\t\n\n\n\t\n\n\t\n\t\t...,malicious
2,"<!DOCTYPE html>\n<html lang=""en"">\n <head>\...",malicious
3,"<!DOCTYPE html><html lang=""en""><head><style da...",malicious
4,<!DOCTYPE html>\n\n\n \n \n \n \n ...,malicious


---
# <b>Step 1. 데이터 수집

### 주어진 데이터로만 모델링 하는 경우는 드뭅니다.
### 주어진 데이터 외 추가로 데이터를 수집 또는 생성해야 하는 경우가 많습니다.
### 이번 과정에서는 웹 크롤러를 통해 수집된 정상/악성 사이트 HTML 데이터에서
### BeatifulSoup 라이브러리를 활용 필요한 Feature(특징)를 추출해 보도록 하겠습니다.
### 정상/악성 사이트 HTML Code는 사전에 수집하여 'Feature Website.xlsx' 파일에 저장해 두었습니다.


### <span style="color:blue">[예시] Beatuifulsoup 라이브러리를 활용 HTML code를 출력하고 \<title> 태그 길이를 계산합니다.<span>

In [153]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(df['html_code'][0], 'html.parser')

*<span style="color:blue"> html code 출력<span>

In [154]:
print(soup)

<!DOCTYPE html>

<!--[if lt IE 7]> <html lang="en-us" class="a-no-js a-lt-ie9 a-lt-ie8 a-lt-ie7"> <![endif]-->
<!--[if IE 7]>    <html lang="en-us" class="a-no-js a-lt-ie9 a-lt-ie8"> <![endif]-->
<!--[if IE 8]>    <html lang="en-us" class="a-no-js a-lt-ie9"> <![endif]-->
<!--[if gt IE 8]><!-->
<html class="a-no-js" lang="en-us"><!--<![endif]--><head>
<meta content="text/html; charset=utf-8" http-equiv="content-type"/>
<meta charset="utf-8"/>
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
<title dir="ltr">Amazon.com</title>
<meta content="width=device-width" name="viewport"/>
<link href="https://images-na.ssl-images-amazon.com/images/G/01/AUIClients/AmazonUI-3c913031596ca78a3768f4e934b1cc02ce238101.secure.min._V1_.css" rel="stylesheet"/>
<script>

if (true === true) {
    var ue_t0 = (+ new Date()),
        ue_csm = window,
        ue = { t0: ue_t0, d: function() { return (+new Date() - ue_t0); } },
        ue_furl = "fls-na.amazon.com",
        ue_mid = "ATVPDKIKX0DER"

* <span style="color:blue"> \<title> 태그 출력 및 길이 계산<span>

In [155]:
# <title> 태그 출력
print("* title :",soup.head.title)

# <title> 태그 길이 출력
print("* title 길이 :", len(str(soup.head.title.getText())))

* title : <title dir="ltr">Amazon.com</title>
* title 길이 : 10


In [156]:
def title_length(soup):
    try:
        return(len(str(soup.head.title.getText())))
    except:
        return 0.0

In [157]:
title_len = []

for index, row in df.iterrows():
    soup = BeautifulSoup(row.html_code, 'html.parser')
    title_len.append(title_length(soup))

In [158]:
df['title_length'] = title_len
df.head()

,html_code,repu,title_length
0,<!DOCTYPE html>\n<!--[if lt IE 7]> <html lang=...,malicious,10.0
1,\n\t\n\n\n\t\n\n\t\n\n\n\t\n\n\n\t\n\n\t\n\t\t...,malicious,5.0
2,"<!DOCTYPE html>\n<html lang=""en"">\n <head>\...",malicious,71.0
3,"<!DOCTYPE html><html lang=""en""><head><style da...",malicious,0.0
4,<!DOCTYPE html>\n\n\n \n \n \n \n ...,malicious,27.0


---

## <b>Q2. html 에서 \<script>...\</script> 태그 길이 계산
- BeautifulSoup으로 html소스를 python 객체로 변환
- 함수로 구현하기
- float으로 return 받기

### <span style="color:darkred">[문제2] Beatuifulsoup 라이브러리를 활용 HTML code에서 \<script> 태그 길이를 계산하는 함수를 완성하고 결과를 확인하세요.<span>

In [159]:
# Feature(특징) 데이터를 추출는 함수를 작성합니다.

def script_tag_len(soup):
    try:
        return(float(len(str(soup.html.script))))
    except:
            return 0.0

In [160]:
# 데이터 프레임의 html_code 컬럼에서 Feature(특징) 데이터를 추출합니다.

script_len = []

for index, row in df.iterrows():
    soup = BeautifulSoup(row.html_code, 'html.parser')
    script_len.append(script_tag_len(soup))

In [161]:
# 추출한 Feature(특징) 데이터를 확인합니다.
df['script_length'] = script_len
df.head()

,html_code,repu,title_length,script_length
0,<!DOCTYPE html>\n<!--[if lt IE 7]> <html lang=...,malicious,10.0,405.0
1,\n\t\n\n\n\t\n\n\t\n\n\n\t\n\n\n\t\n\n\t\n\t\t...,malicious,5.0,579.0
2,"<!DOCTYPE html>\n<html lang=""en"">\n <head>\...",malicious,71.0,817.0
3,"<!DOCTYPE html><html lang=""en""><head><style da...",malicious,0.0,4.0
4,<!DOCTYPE html>\n\n\n \n \n \n \n ...,malicious,27.0,425.0


---

## <b>Q3. html에서 공백 수 계산

- BeautifulSoup으로 html소스를 python 객체로 변환
- 함수로 구현하기
- float으로 return 받기

### <span style="color:darkred">[문제3] Beatuifulsoup 라이브러리를 활용 HTML Code에서 \<html> 태그 공백 수를 계산하는 함수를 완성하고 결과를 확인하세요.<span>

In [162]:
# Feature(특징) 데이터를 추출하는 함수를 작성합니다.

def html_blank_lengh(soup):
    try:
        return float(str(soup.html).count(" "))
    except:
            return 0.0

In [163]:
# 데이터 프레임의 html_code 컬럼에서 Feature(특징) 데이터를 추출합니다.

html_blank_len = []

for index, row in df.iterrows():
    soup = BeautifulSoup(row.html_code, 'html.parser')
    html_blank_len.append(html_blank_lengh(soup))

In [164]:
# 추출한 Feature(특징) 데이터를 확인합니다.
df['html_blank_lengt'] = html_blank_len
df.head()

,html_code,repu,title_length,script_length,html_blank_lengt
0,<!DOCTYPE html>\n<!--[if lt IE 7]> <html lang=...,malicious,10.0,405.0,471.0
1,\n\t\n\n\n\t\n\n\t\n\n\n\t\n\n\n\t\n\n\t\n\t\t...,malicious,5.0,579.0,1318.0
2,"<!DOCTYPE html>\n<html lang=""en"">\n <head>\...",malicious,71.0,817.0,2718.0
3,"<!DOCTYPE html><html lang=""en""><head><style da...",malicious,0.0,4.0,197.0
4,<!DOCTYPE html>\n\n\n \n \n \n \n ...,malicious,27.0,425.0,2771.0


---

## <b>Q4. html 에서 body 길이 계산

- BeautifulSoup으로 html소스를 python 객체로 변환
- 함수로 구현하기
- float으로 return 받기

### <span style="color:darkred">[문제4] Beatuifulsoup 라이브러리를 활용 HTML code에서 \<body> 태그 길이를 계산하는 함수를 완성하고 결과를 확인하세요.<span>

In [165]:
# Feature(특징) 데이터를 추출하는 함수를 작성합니다.

def body_length(soup):
    try:
        return float((len(str(soup.body))))
    except:
            return 0.0    

In [166]:
# 데이터 프레임의 html_code 컬럼에서 Feature(특징) 데이터를 추출합니다.
body_len = []

for index, row in df.iterrows():
    soup = BeautifulSoup(row.html_code, 'html.parser')
    body_len.append(body_length(soup))

In [167]:
# 추출한 Feature(특징) 데이터를 확인합니다.
df['body_length'] = body_len
df.head()

,html_code,repu,title_length,script_length,html_blank_lengt,body_length
0,<!DOCTYPE html>\n<!--[if lt IE 7]> <html lang=...,malicious,10.0,405.0,471.0,3836.0
1,\n\t\n\n\n\t\n\n\t\n\n\n\t\n\n\n\t\n\n\t\n\t\t...,malicious,5.0,579.0,1318.0,15268.0
2,"<!DOCTYPE html>\n<html lang=""en"">\n <head>\...",malicious,71.0,817.0,2718.0,16542.0
3,"<!DOCTYPE html><html lang=""en""><head><style da...",malicious,0.0,4.0,197.0,4.0
4,<!DOCTYPE html>\n\n\n \n \n \n \n ...,malicious,27.0,425.0,2771.0,20314.0


---

## <b>Q5. script 에서 src, href 속성을 가진 태그수

- BeautifulSoup으로 html소스를 python 객체로 변환
- 함수로 구현하기
- float으로 return 받기

### <span style="color:darkred">[문제5] Beatuifulsoup 라이브러리를 활용 HTML code에서 \<script> 태그에서 src, href 속성을 가진 태그수를 계산하는 함수를 완성하고 결과를 확인하세요. <span>


In [168]:
# Feature(특징) 데이터를 추출하는 함수를 작성합니다.
def script_inner_count_src_href(soup):
    try:
        count = 0
        script_elements = soup.find_all('script')
        for el in script_elements:
            if el.get('href') != None:
                count += 1
            if el.get('src') != None:
                count += 1
        return float(count)
    except:
            return 0.0    
        
# def link_in_script(soup):
#     Linkcount = len(soup.findAll('script', {"src" : True}))
#     Linkcount += len(soup.findAll('script', {"href" : True}))
#     return float(Linkcount)

In [169]:
# 데이터 프레임의 html_code 컬럼에서 Feature(특징) 데이터를 추출합니다
script_inner_src_href_count_len = []

for index, row in df.iterrows():
    soup = BeautifulSoup(row.html_code, 'html.parser')
    script_inner_src_href_count_len.append(script_inner_count_src_href(soup))

In [170]:
# 추출한 Feature(특징) 데이터를 확인합니다.
df['script_inner_src_href_count_lengt'] = script_inner_src_href_count_len
df.head()

,html_code,repu,title_length,script_length,html_blank_lengt,body_length,script_inner_src_href_count_lengt
0,<!DOCTYPE html>\n<!--[if lt IE 7]> <html lang=...,malicious,10.0,405.0,471.0,3836.0,0.0
1,\n\t\n\n\n\t\n\n\t\n\n\n\t\n\n\n\t\n\n\t\n\t\t...,malicious,5.0,579.0,1318.0,15268.0,4.0
2,"<!DOCTYPE html>\n<html lang=""en"">\n <head>\...",malicious,71.0,817.0,2718.0,16542.0,8.0
3,"<!DOCTYPE html><html lang=""en""><head><style da...",malicious,0.0,4.0,197.0,4.0,0.0
4,<!DOCTYPE html>\n\n\n \n \n \n \n ...,malicious,27.0,425.0,2771.0,20314.0,0.0


## <b>Q6. 추가적으로 도출 가능한 Feature

- BeautifulSoup으로 html소스를 python 객체로 변환
- 함수로 구현하기
- 적절한 자료형으로 return 받기

### <span style="color:darkred">[문제6] Beatuifulsoup 라이브러리를 활용 HTML code에서 추가로 만들수 있는 Feature를 찾아보고 결과를 확인하세요. <span>


In [171]:
# Feature(특징) 데이터를 추출하는 함수를 작성합니다.


In [172]:
# 데이터 프레임의 html_code 컬럼에서 Feature(특징) 데이터를 추출합니다


In [173]:
# 추출한 Feature(특징) 데이터를 확인합니다.

